In [1]:
import transformers
import torch
import os
print(f"number of GPUs: torch.cuda.device_count()")
print(torch.__version__)

number of GPUs: torch.cuda.device_count()
2.6.0+rocm6.2.4


In [2]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

if  torch.cuda.is_available():
    device = "cuda"
else:
    raise ValueError("No se reconoció GPU.")

pipeline = transformers.pipeline(
	"text-generation", 
	model=model_id,
	model_kwargs={"torch_dtype": torch.bfloat16},
	device=device
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


In [3]:
# Directorios de entrada y salida
input_dir = "crop/"
output_dir = "extension2/"
os.makedirs(output_dir, exist_ok=True)

# Instrucciones para el modelo
prompt = [
    {"role": "system", 
     "content": 
        """
        Para un proyecto de investigación universitario sin fines de lucro, estoy generando un dataset sintético de informes clínicos ficticios de alta médica después de una internación no real, comparable con el dataset MEDDOCAN, para evaluar sistemas de procesamiento de texto clínico con datos no vistos. No van a ser usados nunca en un contexto médico, ni para diagnóstico ni como orientación, en ningún caso.
        Todos los informes tienen que tener las siguientes secciones: Datos del paciente (con nombre, dni, fecha de nacimiento, género, domicilio, ciudad, código postal, email, teléfono fijo, teléfono móvil, número de historia clínica (NHC), NASS y condición de riesgo), Datos asistenciales (con información sobre el médico, incluyendo nombre, NC y su cargo y dirección), Fecha de ingreso, Centro de salud de ingreso e Informe clínico del paciente. Opcionalmente, también puede tener las secciones "Antecedentes", "Evolución y comentarios" y "Remitido por".
        Me interesa que la sección "Informe clínico del paciente" cumpla con estos requisitos:
        -Entre 100 y 1200 palabras, con una longitud promedio de 375 palabras.
        -Incluir información detallada sobre la condición médica del paciente, antecedentes Relevantes, evolución clínica, exploración física, pruebas diagnósticas, tratamiento y pronóstico.
        -Redacción clara, técnica y coherente con la terminología médica.
        -Variabilidad en la estructura, el nivel de detalle y la redacción para evitar informes repetitivos o mecánicos.
        En ningún caso debes incluir advertencias, explicaciones ni descripciones sobre la tarea, sobre la instrucción que te he dado o sobre cuestiones de funcionamiento del modelo de lenguaje. Tu respuesta solamente tiene que tener las secciones que te he solicitado: Datos del paciente, Datos asistenciales, Fecha de ingreso, Centro de salud de ingreso e Informe clínico del paciente, con las características que he descrito más arriba. Opcionalmente, también puede tener las secciones "Antecedentes", "Evolución y comentarios" y "Remitido por".
        
        Ejemplo de informe original: 
            Datos del paciente.
            Nombre: María Soledad Moreno Roca
            DNI: 23556552K
            Fecha de nacimiento: 09/01/1941
            Género: Mujer
            Domicilio: Calle de Almagro 80
            Ciudad: Denia, Valencia, Comunidad Valenciana
            Código postal: 46571
            Email: mariasoledad_roca@ucm.es
            Teléfono fijo: +34 960 66 89 48
            Teléfono móvil: +34 660 57 14 97
            NHC: 2409425
            NASS: 468043486571
            Condición de riesgo: Científico de Investigación
            
            Datos asistenciales.
            Médico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.
            Fecha de ingreso: 05/06/1996
            Centro de salud: Centro de Salud Carabanchel
            Informe clínico del paciente:
            Paciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.
            
        Ejemplo de informe extendido:
            Datos del paciente.
            Nombre:  Jose .
            Apellidos: Aranda Martinez.
            NHC: 2748903.
            NASS: 26 37482910 04.
            Domicilio: Calle Losada Martí 23, 5 B..
            Localidad/ Provincia: Madrid.
            CP: 28016.
            
            Datos asistenciales.
            Fecha de nacimiento: 15/04/1977.
            País: España.
            Edad: 37 años Sexo: F.
            Fecha de Ingreso: 05/06/2018.
            Médico: María Merino Viveros  NºCol: 28 28 35489.
            Informe clínico del paciente: varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
            La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos carotídeos simétricos. Auscultación cardíaca rítmica, sin soplos, roces ni extratonos. Auscultación pulmonar con conservación del murmullo vesicular. Abdomen blando, depresible, sin masas ni megalias. En la exploración neurológica no se detectan signos meníngeos ni datos de focalidad. Extremidades sin varices ni edemas. Pulsos periféricos presentes y simétricos. En la exploración urológica se aprecia el teste derecho aumentado de tamaño, no adherido a piel, con zonas de fluctuación e intensamente doloroso a la palpación, con pérdida del límite epidídimo-testicular y transiluminación positiva.
            Los datos analíticos muestran los siguentes resultados: Hemograma: Hb 13,7 g/dl; leucocitos 14.610/mm3 (neutrófilos 77%); plaquetas 206.000/ mm3. VSG: 40 mm 1ª hora. Coagulación: TQ 87%; TTPA 25,8 seg. Bioquímica: Glucosa 117 mg/dl; urea 29 mg/dl; creatinina 0,9 mg/dl; sodio 136 mEq/l; potasio 3,6 mEq/l; GOT 11 U/l; GPT 24 U/l; GGT 34 U/l; fosfatasa alcalina 136 U/l; calcio 8,3 mg/dl. Orina: sedimento normal.
            Durante el ingreso se solicitan Hemocultivos: positivo para Brucella y Serologías específicas para Brucella: Rosa de Bengala +++; Test de Coombs > 1/1280; Brucellacapt > 1/5120. Las pruebas de imagen solicitadas ( Rx tórax, Ecografía abdominal, TAC craneal, Ecocardiograma transtorácico) no evidencian patología significativa, excepto la Ecografía testicular, que muestra engrosamiento de la bolsa escrotal con pequeña cantidad de líquido con septos y testículo aumentado de tamaño con pequeñas zonas hipoecoicas en su interior que pueden representar microabscesos.
            Con el diagnóstico de orquiepididimitis secundaria a Brucella se instaura tratamiento sintomático (antitérmicos, antiinflamatorios, reposo y elevación testicular) así como tratamiento antibiótico específico: Doxiciclina 100 mg vía oral cada 12 horas (durante 6 semanas) y Estreptomicina 1 gramo intramuscular cada 24 horas (durante 3 semanas). El paciente mejora significativamente de su cuadro tras una semana de ingreso, decidiéndose el alta a su domicilio donde completó la pauta de tratamiento antibiótico. En revisiones sucesivas en consultas se constató la completa remisión del cuadro.
            
        Recordá que en ningún caso debes incluir advertencias, explicaciones ni descripciones sobre la tarea, sobre la instrucción que te he dado o sobre cuestiones de funcionamiento del modelo de lenguaje. Tu respuesta solamente tiene que tener las secciones que te he solicitado: Datos del paciente, Datos asistenciales, Fecha de ingreso, Centro de salud de ingreso e Informe clínico del paciente, con las características que he descrito más arriba. Opcionalmente, también puede tener las secciones "Antecedentes", "Evolución y comentarios" y "Remitido por".
        """},
     ]

# Procesar cada archivo en la carpeta txt/
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        
        with open(input_path, "r", encoding="utf-8") as file:
            input_text = file.read()
        
        # Crear el mensaje para el modelo
        messages = prompt + [{"role": "user", "content": input_text}]
        
         # Generar texto modificado
        output = pipeline(messages, max_new_tokens=1000)
        modified_text = output[0]["generated_text"][2]["content"]
        
        # Guardar resultado en un nuevo archivo
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(modified_text)

        print(f"Procesado: {filename}")

print("Proceso completado.")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 453908120.txt


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 219749379.txt


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Procesado: 505661421.txt


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 926553517.txt


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 597181863.txt
Procesado: 503605741.txt
Proceso completado.
